In [170]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb

In [171]:
def clean(ticker):
    ydf = yf.download(ticker, interval="1d")
    #ydf.to_csv(f'y{ticker}.csv')
    ydf.to_csv(f'test.csv')

    # btc = pd.read_csv(f'y{ticker}.csv')
    btc = pd.read_csv(f'test.csv')
    btc = btc.drop([0,1])
    btc['Date'] = pd.to_datetime(btc['Price']).dt.date
    # btc['Date'] = pd.to_datetime(btc['Price']).dt.floor('T')
    btc= btc.set_index('Date')
    btc = btc.drop(['Price'], axis = 1)
    btc = btc.reset_index()
    #btc['Ticker'] = ticker
    return btc

In [172]:
btc = clean('BTC-USD')
btc[btc.columns[1:]] = btc[btc.columns[1:]].astype(float)

btc

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,2.105680e+07
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,3.448320e+07
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,3.791970e+07
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,3.686360e+07
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,2.658010e+07
...,...,...,...,...,...,...,...
3703,2024-11-06,75639.078125,75639.078125,76460.156250,69322.031250,69358.500000,1.185927e+11
3704,2024-11-07,75904.859375,75904.859375,76943.117188,74480.421875,75637.085938,6.346765e+10
3705,2024-11-08,76545.476562,76545.476562,77252.750000,75648.742188,75902.835938,5.517686e+10
3706,2024-11-09,76778.867188,76778.867188,76932.765625,75773.789062,76556.187500,2.900948e+10


In [173]:

btc['ret'] = btc['Adj Close'].pct_change()
btc['logret'] = np.log1p(btc['ret'])
btc['Gain'] = btc['logret'].apply(lambda x: x if x > 0 else 0)
btc['Loss'] = btc['logret'].apply(lambda x: -x if x < 0 else 0)

windows = [7, 15, 30, 60, 120, 180]

for i in windows:
    #rolling volatilities
    btc[f'vol_{i}'] = btc['logret'].rolling(window=i).std()

    #relative strength index
    btc[f'RSI_{i}'] = 100 - (100 / (1 + (btc['Gain'].rolling(window=i).mean() / btc['Loss'].rolling(window=i).mean())))

    # momentum score
    btc[f'mom_{i}'] = (1/i) * btc['logret'].rolling(i,i).sum().reset_index(drop=True)

    #exponential moving average
    btc[f'ewma_{i}'] = btc['logret'].ewm(span=i, min_periods=i).mean()

btc = btc.dropna().reset_index(drop=True)
btc['target'] = (btc['logret'].shift(-1) > 0).astype(int)
btc


,Date,Adj Close,Close,High,Low,Open,Volume,ret,logret,Gain,...,ewma_60,vol_120,RSI_120,mom_120,ewma_120,vol_180,RSI_180,mom_180,ewma_180,target
0,2015-03-16,290.592987,290.592987,294.112000,285.684998,285.684998,2.151610e+07,0.014665,0.014559,0.014559,...,0.003347,0.046246,46.023427,-0.002406,0.000585,0.043956,45.790611,-0.002519,-0.000496,0
1,2015-03-17,285.505005,285.505005,292.364990,284.373993,290.595001,2.149720e+07,-0.017509,-0.017664,0.000000,...,0.002656,0.046267,45.815970,-0.002543,0.000268,0.043637,46.280156,-0.002203,-0.000715,0
2,2015-03-18,256.299011,256.299011,285.335999,249.869995,285.066986,5.700800e+07,-0.102296,-0.107915,0.000000,...,-0.000978,0.047183,44.882128,-0.003176,-0.001610,0.044035,45.973818,-0.002400,-0.002080,1
3,2015-03-19,260.928009,260.928009,264.243988,248.636002,255.880005,5.273200e+07,0.018061,0.017900,0.017900,...,-0.000357,0.047196,44.937185,-0.003145,-0.001272,0.043972,45.799945,-0.002496,-0.001826,1
4,2015-03-20,261.748993,261.748993,264.847992,259.161987,260.955994,1.845670e+07,0.003146,0.003141,0.003141,...,-0.000242,0.046891,45.738199,-0.002606,-0.001195,0.043941,46.046612,-0.002340,-0.001763,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,2024-11-06,75639.078125,75639.078125,76460.156250,69322.031250,69358.500000,1.185927e+11,0.090536,0.086669,0.086669,...,0.004868,0.026900,55.699045,0.002211,0.002890,0.025679,53.271906,0.001214,0.002233,1
3524,2024-11-07,75904.859375,75904.859375,76943.117188,74480.421875,75637.085938,6.346765e+10,0.003514,0.003508,0.003508,...,0.004824,0.026893,55.876128,0.002279,0.002900,0.025679,53.320727,0.001233,0.002247,1
3525,2024-11-08,76545.476562,76545.476562,77252.750000,75648.742188,75902.835938,5.517686e+10,0.008440,0.008404,0.008404,...,0.004941,0.026885,56.201170,0.002407,0.002991,0.025675,53.288487,0.001220,0.002315,1
3526,2024-11-09,76778.867188,76778.867188,76932.765625,75773.789062,76556.187500,2.900948e+10,0.003049,0.003044,0.003044,...,0.004879,0.026877,56.077063,0.002352,0.002992,0.025622,53.002483,0.001108,0.002323,1


In [174]:
X = btc[btc.columns[11:-1]]
y = btc['target']

In [175]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)

xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

xgb_predictions = xgb_model.predict(X_test)

# Evaluate Models
print("Random Forest Accuracy:")
print(accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

print("\nXGBoost Accuracy:")
print(accuracy_score(y_test, xgb_predictions))
print(classification_report(y_test, xgb_predictions))


Random Forest Accuracy:
0.5283286118980169
              precision    recall  f1-score   support

           0       0.53      0.42      0.47       350
           1       0.53      0.63      0.57       356

    accuracy                           0.53       706
   macro avg       0.53      0.53      0.52       706
weighted avg       0.53      0.53      0.52       706


XGBoost Accuracy:
0.5070821529745042
              precision    recall  f1-score   support

           0       0.50      0.45      0.47       350
           1       0.51      0.56      0.54       356

    accuracy                           0.51       706
   macro avg       0.51      0.51      0.51       706
weighted avg       0.51      0.51      0.51       706



In [176]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print("Random Forest Results:")
print(f"Mean Squared Error: {rf_mse}")
print(f"R-squared Score: {rf_r2}")


Random Forest Results:
Mean Squared Error: 0.25798399433427766
R-squared Score: -0.03201051524879617


In [177]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

xgb_predictions = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print("\nXGBoost Results:")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared Score: {xgb_r2}")



XGBoost Results:
Mean Squared Error: 0.3101088833373724
R-squared Score: -0.24052512645721436
